<a href="https://www.kaggle.com/code/lovrorabuzin/swiftnet-zavrad?scriptVersionId=91692088" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importi i hiperparametri

In [ ]:
import torch
from torch import nn
from torch.nn import functional
import torch.nn.functional as F

import sklearn
from sklearn import model_selection

import torchvision
import torchvision.transforms as torch_transforms
import torch.utils.data as data
import torch.optim as optim
import torchvision.models as models

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from torch.utils.data import Subset

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

import pickle
import numpy as np
import math
import pandas as pd

import skimage as ski
import skimage.io
import os

import random

import nibabel as nib
from PIL import Image
import imageio

import torch.utils.checkpoint as cp

import sys
import json

from math import e

b_s = 1
learning_rate = 0.0001
scaling_factor = 2
weight_decay = 0.0005
gamma = 0.95
num_epochs = 30
slice_no = 200
random.seed()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
root_train_dir = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'

# Učitavanje podataka

In [ ]:
name_mapping = pd.read_csv(root_train_dir + '/name_mapping.csv')

name_mapping.rename({'BraTS_2020_subject_ID': 'ID'}, axis=1, inplace=True)

survival_info = pd.read_csv(root_train_dir + '/survival_info.csv')

survival_info.rename({'Brats20ID': 'ID'}, axis=1, inplace=True)

patient_info = survival_info.merge(name_mapping, on="ID", how="right")

modalities = ['_flair.nii', '_t1.nii', '_t1ce.nii', '_t2.nii']
mask_path = '_seg.nii'

train_scan_files = []
valid_scan_files = []
test_scan_files = []

HGG_names = list(patient_info[patient_info['Grade'] == 'HGG'].ID)
LGG_names = list(patient_info[patient_info['Grade'] == 'LGG'].ID)

for i in range(len(HGG_names)):
    image_path = root_train_dir+'/'+HGG_names[i]+'/'+HGG_names[i]
    if i % 7 == 5:
        valid_scan_files.append(image_path)
    elif i % 7 == 4:
        test_scan_files.append(image_path)
    else:
        train_scan_files.append(image_path)

for i in range(len(LGG_names)):
    image_path = root_train_dir+'/'+LGG_names[i]+'/'+LGG_names[i]
    if i % 7 == 5:
        valid_scan_files.append(image_path)
    elif i % 7 == 4:
        test_scan_files.append(image_path)
    else:
        train_scan_files.append(image_path)

horizontalFlipTransform = torch_transforms.RandomHorizontalFlip(p = 0.5)
toTensor = torch_transforms.ToTensor()

trainTransform = torch_transforms.Compose([toTensor, horizontalFlipTransform])

def normalize_vol(volume, i):
    logical_mask = volume != 0.
    mean = np.mean(volume[logical_mask])
    std = np.std(volume[logical_mask])
    return (volume-mean)/std

class SegmentationDataset(Dataset):
    in_channels = 4
    out_channels = 4
    
    def __init__(self, paths, modalities, mask_path, tumor_slices, slice_no = 155, transform = None):
        self.paths = paths
        self.modalities = modalities
        self.mask_path = mask_path
        self.transform = transform
        self.mask_buffer = np.array([])
        self.volume_buffer = np.array([])
        self.passes = 0
        self.tumor_slices = tumor_slices
        random.shuffle(self.paths)
        
    def __len__(self):
        return math.ceil(self.tumor_slices/slice_no)
    
    def reset(self):
        self.passes = 0
        random.shuffle(self.paths)
    
    def __getitem__(self, idx):
        if self.mask_buffer.size > 0:
            bound = min(slice_no, self.mask_buffer.shape[0])
            res_mask = self.mask_buffer[:bound]
            res_volume = self.volume_buffer[:bound]
            if bound < self.mask_buffer.shape[0]:
                self.mask_buffer = self.mask_buffer[bound:]
                self.volume_buffer = self.volume_buffer[bound:]
            else:
                self.mask_buffer = np.array([])
                self.volume_buffer = np.array([])
        else:
            res_mask = np.array([])
            res_volume = np.array([])
        
        while res_mask.shape[0] < slice_no and idx + self.passes < len(self.paths):
            patient = self.paths[idx+self.passes]
            self.passes += 1
            volumes = []
            single_mask = patient + mask_path
            if single_mask == root_train_dir + '/BraTS20_Training_355/BraTS20_Training_355_seg.nii':
                single_mask = root_train_dir + '/BraTS20_Training_355/W39_1998.09.19_Segm.nii'
            single_mask = nib.load(single_mask)
            single_mask = np.asarray(single_mask.dataobj, dtype = np.float)
            mask = single_mask.transpose(2,0,1)
            mask_WT = mask.copy()
            mask_WT[mask_WT == 1] = 0
            mask_WT[mask_WT == 2] = 1
            mask_WT[mask_WT == 4] = 0

            mask_TC = mask.copy()
            mask_TC[mask_TC == 1] = 1
            mask_TC[mask_TC == 2] = 0
            mask_TC[mask_TC == 4] = 0

            mask_ET = mask.copy()
            mask_ET[mask_ET == 1] = 0
            mask_ET[mask_ET == 2] = 0
            mask_ET[mask_ET == 4] = 1

            mask_BG = mask.copy()
            mask_BG[mask_BG == 0] = 3
            mask_BG[mask_BG == 1] = 0
            mask_BG[mask_BG == 2] = 0
            mask_BG[mask_BG == 4] = 0
            mask_BG[mask_BG == 3] = 1

            mask_full = np.stack([mask_WT, mask_TC, mask_ET, mask_BG])
            mask_full = np.transpose(mask_full, (1,0,2,3))

            tumor_indices = []
            mask_pure = []
            for i in range(155):
                mask_slice = mask_full[i]
                if np.sum(mask_slice[0:3]) != 0:
                    tumor_indices.append(i)
                    mask_pure.append(mask_slice)
            mask = np.stack(mask_pure)

            for modality in modalities:
                single_mod_volume = patient + modality
                single_mod_volume = nib.load(single_mod_volume)
                single_mod_volume = np.asarray(single_mod_volume.dataobj, dtype = np.float)
                single_mod_volume = single_mod_volume.transpose(2,0,1)
                intermittent = []
                for i_s in tumor_indices:
                    intermittent.append(single_mod_volume[i_s])
                volumes.append(np.stack(intermittent))

            if self.transform:
                seed = random.randint(0,2**32)
                random.seed(seed)
                torch.manual_seed(seed)
                volumes = np.transpose(np.stack(volumes), (1,0,2,3))
                volumes = np.stack([self.transform(np.transpose(volumes[i], (1,2,0))) for i in range(np.shape(volumes)[0])])
                volumes = np.transpose(np.stack(volumes), (1,0,2,3))
                random.seed(seed)
                torch.manual_seed(seed)
                mask = np.stack([self.transform(np.transpose(mask[i], (1,2,0))) for i in range(np.shape(mask)[0])])
            volumes = [normalize_vol(volumes[i], i) for i in range(np.shape(volumes)[0])]
            volumes = np.transpose(np.stack(volumes), (1,0,2,3))
            bound = min(slice_no-res_mask.shape[0], mask.shape[0])
            if res_mask.size == 0:
                res_mask = mask[:bound]
                res_volume = volumes[:bound]
            else:
                res_mask = np.concatenate((res_mask, mask[:bound]), axis = 0)
                res_volume = np.concatenate((res_volume, volumes[:bound]), axis = 0)
            if bound < mask.shape[0]:
                if self.mask_buffer.size == 0:
                    self.mask_buffer = mask[bound:]
                    self.volume_buffer = volumes[bound:]
                else:
                    self.mask_buffer = np.concatenate((self.mask_buffer, mask[bound:]), axis = 0)
                    self.volume_buffer = np.concatenate((self.volume_buffer, volumes[bound:]), axis = 0)
        self.passes -= 1
        res_volume = torch.from_numpy(res_volume).float()
        res_mask = torch.from_numpy(res_mask).long()
        return res_volume, res_mask
    
class TestingDataset(Dataset):
    in_channels = 4
    out_channels = 4
    
    def __init__(self, paths, modalities, mask_path):
        self.paths = paths
        self.modalities = modalities
        self.mask_path = mask_path
        self.mask_buffer = np.array([])
        self.volume_buffer = np.array([])
        
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        patient = self.paths[idx]
        volumes = []
        single_mask = patient + mask_path
        if single_mask == root_train_dir + '/BraTS20_Training_355/BraTS20_Training_355_seg.nii':
            single_mask = root_train_dir + '/BraTS20_Training_355/W39_1998.09.19_Segm.nii'
        single_mask = nib.load(single_mask)
        single_mask = np.asarray(single_mask.dataobj, dtype = np.float)
        mask = single_mask.transpose(2,0,1)
        mask_WT = mask.copy()
        mask_WT[mask_WT == 1] = 0
        mask_WT[mask_WT == 2] = 1
        mask_WT[mask_WT == 4] = 0

        mask_TC = mask.copy()
        mask_TC[mask_TC == 1] = 1
        mask_TC[mask_TC == 2] = 0
        mask_TC[mask_TC == 4] = 0

        mask_ET = mask.copy()
        mask_ET[mask_ET == 1] = 0
        mask_ET[mask_ET == 2] = 0
        mask_ET[mask_ET == 4] = 1

        mask_BG = mask.copy()
        mask_BG[mask_BG == 0] = 3
        mask_BG[mask_BG == 1] = 0
        mask_BG[mask_BG == 2] = 0
        mask_BG[mask_BG == 4] = 0
        mask_BG[mask_BG == 3] = 1
        mask_full = np.stack([mask_WT, mask_TC, mask_ET, mask_BG])#, mask_BG
        mask_full = np.transpose(mask_full, (1,0,2,3))
        
        for modality in modalities:
                single_mod_volume = patient + modality
                single_mod_volume = nib.load(single_mod_volume)
                single_mod_volume = np.asarray(single_mod_volume.dataobj, dtype = np.float)
                single_mod_volume = single_mod_volume.transpose(2,0,1)
                volumes.append(np.stack(single_mod_volume))

        volumes = [normalize_vol(volumes[i], i) for i in range(np.shape(volumes)[0])]
        volumes = np.transpose(np.stack(volumes), (1,0,2,3))
        res_volume = torch.from_numpy(volumes).float()
        res_mask = torch.from_numpy(mask_full).long()

        return res_volume, res_mask



trainset = SegmentationDataset(train_scan_files, modalities, mask_path, 17227, transform = trainTransform)
validset = SegmentationDataset(valid_scan_files, modalities, mask_path, 3661)
testset = TestingDataset(test_scan_files, modalities, mask_path)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=b_s, shuffle=False, num_workers=0)
validloader = torch.utils.data.DataLoader(validset, batch_size=b_s,shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=b_s, shuffle=False, num_workers=0)

# Neuronska mreža (SwiftNet)

In [ ]:
upsample = lambda x, size: F.interpolate(x, size, mode='bilinear', align_corners=False)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, channels, stride=1, identity_downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size = 3, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(channels)
        self.downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        residual = x.clone()
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample is not None:
            residual = self.downsample(residual)
        x += residual
        relu = self.relu(x)
        return relu, x


class BNReLUConv(nn.Sequential):
    def __init__(self, in_channels, out_channels, k=3):
        super(BNReLUConv, self).__init__()
        self.add_module('BN', nn.BatchNorm2d(in_channels))
        self.add_module('ReLU', nn.ReLU())
        padding = k // 2
        self.add_module('Conv', nn.Conv2d(in_channels, out_channels, k, padding=padding, bias=False))

        
class Up(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels, k=3):
        super(Up, self).__init__()
        self.bottleneck = BNReLUConv(skip_channels, in_channels, k = 1)
        self.blend = BNReLUConv(in_channels, out_channels, k = k)
        self.upsample = lambda x, size: F.interpolate(x, size, mode='bilinear', align_corners=False)
    
    def forward(self, x, skip):
        skip = self.bottleneck.forward(skip)
        skip_size = skip.size()[2:4]
        x = self.upsample(x, skip_size)
        x = x + skip
        x = self.blend.forward(x)
        return x


class SPP(nn.Module):
    def __init__(self, in_channels, num_levels, bt_size, level_size, out_size, grids):
        super(SPP, self).__init__()
        self.upsample = lambda x, size: F.interpolate(x, size, mode='bilinear', align_corners=False)
        self.spp = nn.Sequential()
        self.spp.add_module('spp_bn', BNReLUConv(in_channels, bt_size, k=1))
        self.grids = grids
        num_features = bt_size
        final_size = num_features
        for i in range(num_levels):
            final_size += level_size
            self.spp.add_module('spp' + str(i), BNReLUConv(num_features, level_size, k=1))
        self.spp.add_module('spp_fuse', BNReLUConv(final_size, out_size, k=1))
        
    def forward(self, x):
        levels = []
        target_size = x.size()[2:4]
        x = self.spp[0].forward(x)
        levels.append(x)
        num = len(self.spp) - 1
        for i in range(1, num):
            x_pooled = F.adaptive_avg_pool2d(x, self.grids[i - 1])
            level = self.spp[i].forward(x_pooled)
            level = self.upsample(level, target_size)
            levels.append(level)
        x = torch.cat(levels, 1)
        x = self.spp[-1].forward(x)
        return x


class ResNet(nn.Module):
    def __init__(self, residual_block, layers, image_channels=4):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.num_features = 128
        self.conv1 = nn.Conv2d(4, 64, kernel_size = 7, stride = 2, padding = 3, bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        upsamples = []
        self.layer1 = self._make_layer(residual_block, 64, layers[0])
        upsamples += [Up(128, 64, 128)]
        self.layer2 = self._make_layer(residual_block, 128, layers[1], stride=2)
        upsamples += [Up(128, 128, 128)]
        self.layer3 = self._make_layer(residual_block, 256, layers[2], stride=2)
        upsamples += [Up(128, 256, 128)]
        self.layer4 = self._make_layer(residual_block, 512, layers[3], stride=2)
        
        num_levels = 3
        self.spp_size = 128
        bt_size = self.spp_size
        level_size = self.spp_size // num_levels
        self.spp = SPP(self.inplanes, num_levels, bt_size=bt_size, level_size=level_size, out_size=128, grids = (8,4,2,1))
        self.upsample = nn.ModuleList(list(reversed(upsamples)))
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward_resblock(self, x, layers):
        skip = None
        for l in layers:
            x = l(x)
            if isinstance(x, tuple):
                x, skip = x
        return x, skip

    def forward_down(self, image):
        x = self.conv1(image)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        features = []
        x, skip = self.forward_resblock(x, self.layer1)
        features += [skip]
        x, skip = self.forward_resblock(x, self.layer2)
        features += [skip]
        x, skip = self.forward_resblock(x, self.layer3)
        features += [skip]
        x, skip = self.forward_resblock(x, self.layer4)
        features += [self.spp.forward(skip)]
        return features

    def forward_up(self, features):
        features = features[::-1]
        x = features[0]
        upsamples = []
        for skip, up in zip(features[1:], self.upsample):
            x = up(x, skip)
            upsamples += [x]
        return x, {'features': features, 'upsamples': upsamples}

    def forward(self, image):
        return self.forward_up(self.forward_down(image))
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            layers = [nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False)]
            layers += [nn.BatchNorm2d(planes * block.expansion)]
            downsample = nn.Sequential(*layers)
        layers = [block(self.inplanes, planes, stride, downsample)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers += [block(self.inplanes, planes)]
        return nn.Sequential(*layers)

    
class SemSegModel(nn.Module):
    def __init__(self, backbone, num_classes):
        super(SemSegModel, self).__init__()
        self.backbone = backbone
        self.num_classes = num_classes
        self.logits = BNReLUConv(backbone.num_features, self.num_classes)
        self.multiscale_factors=(.5, .75, 1.5, 2.)
        
    def forward(self, image):
        image_size = image.size()[2:4]
        features, additional = self.backbone(image)
        logits = self.logits.forward(features)
        logits = upsample(logits, image_size)
        additional['logits'] = logits
        nonlin = nn.LogSoftmax(1)
        return nonlin(logits)

# Pomoćne funkcije

In [ ]:
def dice_score(prediction, ground_truth, smooth = 1.0):
    prediction = prediction.contiguous().view(-1)
    ground_truth = ground_truth.contiguous().view(-1)
    
    intersection = (prediction*ground_truth).sum()
    score = (2*intersection+smooth)/(prediction.sum()+ground_truth.sum()+smooth)
    return score

def un_one_hot(targets):
    return targets.argmax(1)

def output_metrics(loss_avg, dice_scores, phase):
    print(phase)
    print("Average loss: {}".format(loss_avg))
    print("Dice score: {}".format(dice_scores))

def dataset_evaluate(model, loader, loss_fn, threshold = -1, single = False):
    global device
    
    loss_avg = 0.0
    dice_scores = []
    
    with torch.no_grad():
        model.eval()
        for images, ground_truths in loader:
            images = torch.cat([images[i] for i in range(b_s)])
            ground_truths = torch.cat([ground_truths[i] for i in range(b_s)])
            images = images.to(device)
            ground_truths = ground_truths.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, un_one_hot(ground_truths))
            loss_avg += loss.item()
            outputs = outputs.argmax(1)
            ground_truths = ground_truths.argmax(1)
            outputs_WT = outputs.clone()
            outputs_WT[outputs_WT == 0] = 1
            outputs_WT[outputs_WT == 1] = 1
            outputs_WT[outputs_WT == 2] = 1
            outputs_WT[outputs_WT == 3] = 0
            outputs_TC = outputs.clone()
            outputs_TC[outputs_TC == 0] = 0
            outputs_TC[outputs_TC == 1] = 1
            outputs_TC[outputs_TC == 2] = 1
            outputs_TC[outputs_TC == 3] = 0
            outputs_ET = outputs.clone()
            outputs_ET[outputs_ET == 0] = 0
            outputs_ET[outputs_ET == 1] = 0
            outputs_ET[outputs_ET == 2] = 1
            outputs_ET[outputs_ET == 3] = 0
            ground_truths_WT = ground_truths.clone()
            ground_truths_WT[ground_truths_WT == 0] = 1
            ground_truths_WT[ground_truths_WT == 1] = 1
            ground_truths_WT[ground_truths_WT == 2] = 1
            ground_truths_WT[ground_truths_WT == 3] = 0
            ground_truths_TC = ground_truths.clone()
            ground_truths_TC[ground_truths_TC == 0] = 0
            ground_truths_TC[ground_truths_TC == 1] = 1
            ground_truths_TC[ground_truths_TC == 2] = 1
            ground_truths_TC[ground_truths_TC == 3] = 0
            ground_truths_ET = ground_truths.clone()
            ground_truths_ET[ground_truths_ET == 0] = 0
            ground_truths_ET[ground_truths_ET == 1] = 0
            ground_truths_ET[ground_truths_ET == 2] = 1
            ground_truths_ET[ground_truths_ET == 3] = 0
            
            dice_scores.append([dice_score(outputs_WT, ground_truths_WT),
                                dice_score(outputs_TC, ground_truths_TC),
                                dice_score(outputs_ET, ground_truths_ET)])
                
    loss_avg /= len(loader)
    model.train()
    dice_scores = np.stack(dice_scores)
    return loss_avg, dice_scores.mean(0)

# Iscrtavanje grafa gubitka

In [ ]:
def plot_progress(data):
    valid_loss = data['valid loss']
    train_loss = data['train loss']

    fig, ax = plt.subplots(figsize=(16,8))
    linewidth = 2
    legend_size = 10
    train_color = 'm'
    val_color = 'c'

    ax.set_title('Loss')
    ax.plot(train_loss, marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
    ax.plot(valid_loss, marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
    ax.legend(loc='upper right', fontsize=legend_size)

    save_path = os.path.join('./', 'loss.png')
    print('Plotting in: ', save_path)
    plt.savefig(save_path)
    return

# Treniranje i evaluacija modela

In [ ]:
def train_network():
    global device
    plot_data = {}
    plot_data["train loss"] = []
    plot_data["valid loss"] = []
    plot_data["WT valid dice"] = []
    plot_data["TC valid dice"] = []
    plot_data["ET valid dice"] = []
    plot_data["test loss"] = 0
    plot_data["WT test dice"] = []
    plot_data["TC test dice"] = []
    plot_data["ET test dice"] = []
    plot_data["lr"] = []
    
    SAVE_PATH = "./network.pt"
    print("device:", device)
    
    resnet18 = models.resnet18(pretrained=True)
    myresnet = ResNet(BasicBlock, [2,2,2,2])

    pretrained_dict = resnet18.state_dict()
    model_dict = myresnet.state_dict()

    conv_weight = pretrained_dict.pop('conv1.weight')
    mean_kernel = conv_weight.mean(dim=1)
    mean_kernel = torch.unsqueeze(mean_kernel, dim=1)
    mean_kernel = torch.cat([mean_kernel]*4, dim = 1)
    pretrained_dict['conv1.weight'] = mean_kernel

    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    pretrained_names = list(pretrained_dict.keys())
    model_dict.update(pretrained_dict)

    myresnet.load_state_dict(model_dict)
    
    net = SemSegModel(myresnet, 4).to(device = device)
    lossFunc = nn.NLLLoss(weight = torch.tensor([2,5,3,1]).float().to(device))
    
    base_params = [v[1] for v in list(filter(lambda kv: kv[0] not in pretrained_names, net.named_parameters()))]
    pretrained_params = [v[1] for v in list(filter(lambda kv: kv[0] in pretrained_names, net.named_parameters()))]
    
    optimizer = optim.Adam([{'params': base_params}, {'params': pretrained_params, 'lr':learning_rate/scaling_factor}], lr=learning_rate, weight_decay = weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = num_epochs, eta_min = 1e-6)
    
    for e in range(num_epochs):
        accLoss = 0.
        net.train()
        i = -1
        for inputs, ground_truths in trainloader:
            i+=1
            inputs = torch.cat([inputs[i] for i in range(b_s)])
            ground_truths = torch.cat([ground_truths[i] for i in range(b_s)])
            inputs.requires_grad = True
            inputs = inputs.to(device)
            ground_truths = ground_truths.to(device)
            
            outputs = net(inputs)
            loss = lossFunc(outputs, un_one_hot(ground_truths))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            accLoss += loss.item()

            if i % 10 == 0:
                print("Epoch: %d, Iteration: %5d, Loss: %.3f" % ((e + 1), (i+1), (accLoss / (i + 1))))
        trainset.reset()

        val_loss, val_dice = dataset_evaluate(net, validloader, lossFunc)
        validset.reset()
        output_metrics(val_loss, val_dice, "Validation")
        plot_data["valid loss"].append(val_loss)
        plot_data["WT valid dice"].append(val_dice[0])
        plot_data["TC valid dice"].append(val_dice[1])
        plot_data["ET valid dice"].append(val_dice[2])
        
        plot_data["lr"].append(scheduler.get_last_lr())
        plot_data["train loss"].append(accLoss/(i+1))
        
        scheduler.step()
    
    test_loss, test_dice = dataset_evaluate(net, testloader, lossFunc)
    output_metrics(test_loss, test_dice, "Test:")
    
    plot_data["test loss"] = test_loss
    plot_data["WT test dice"] = test_dice[0]
    plot_data["TC test dice"] = test_dice[1]
    plot_data["ET test dice"] = test_dice[2]
    
    torch.save(net.state_dict(), SAVE_PATH)
    
    with open("./epoch_data.txt", 'w') as f:
        f.write(repr(plot_data))

    plot_progress(plot_data)

    return plot_data
    
    
epoch_data = train_network()